# Structure of dynamic smart vampires

In [1]:
from tqdm import tqdm

import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

from agent.Vampire import SimpleVampire, SmartVampire, SmartDynamicVampire
from agent.Vampire import SMART_VAMPIRE_STRATEGIES_PROB
from model.VampireModel import VampireModel 

In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be re

In [2]:
def map_motivation(value):
    if value < -2:  
        return 'Cheater'
    elif -2 <= value < 0:
        return 'Prudent'
    elif 0 <= value <= 1:
        return 'Fair'
    elif 1 < value <= 4:
        return 'Generous'
    elif value > 4:
        return 'Martyr'

In [3]:
res = []

for _ in tqdm(range(100)):
    model = VampireModel(
        n_roots = 10,
        root_size = 15,
        max_iteration = 5000, 
        hunt_probability = 0.9,
        food_sharing = True,
        reproduction = True,
        reproduction_probability = 0.5,
        vampire_type = SmartDynamicVampire,
        min_motivation = -4,
        max_motivation = 8
    )
    model.run_model()

    # df = model.datacollector.get_model_vars_dataframe()
    # df.plot()

    df = model.datacollector.get_agent_vars_dataframe()
    df['Motivation'] = df['Motivation'].apply(map_motivation)
    df = df.reset_index().groupby(['Motivation', 'Step'])['AgentID'].count().unstack(0)
    res.append(df)

100%|██████████| 100/100 [05:40<00:00,  3.41s/it]


In [4]:
df = pd.concat(res).reset_index().groupby('Step')[['Cheater', 'Fair', 'Generous', 'Martyr', 'Prudent']].mean()

In [5]:
xlabel="Iterations"
ylabel="Number of vampires"
legend1="Cheater"
legend2="Fair"
legend3="Generous"
legend4="Martyr"
legend5="Prudent"

layout = go.Layout(
    xaxis=dict(
        title=xlabel, 
        showgrid=False  
    ),
    yaxis=dict(
        title=ylabel,
        showgrid=False,
    ))

fig = go.Figure([
    go.Scatter(
        name=legend1,
        x=df.index,
        y=df[legend1],
        mode='lines',
        line=dict(color='#FA433E', width=3),
    ),
    go.Scatter(
        name=legend5,
        x=df.index,
        y=df[legend5],
        mode='lines',
        line=dict(color='#DBBEBD', width=3),
    ),
    go.Scatter(
        name=legend2,
        x=df.index,
        y=df[legend2],
        mode='lines',
        line=dict(color='#F5FFF8', width=3),
    ),
    go.Scatter(
        name=legend3,
        x=df.index,
        y=df[legend3],
        mode='lines',
        line=dict(color='#8ADBA6', width=3),
    ),
    go.Scatter(
        name=legend4,
        x=df.index,
        y=df[legend4],
        mode='lines',
        line=dict(color='#4DB070', width=3),
    )
],
    layout=layout
)

fig.update_layout(template="plotly_dark", plot_bgcolor='rgb(0,0,0)', paper_bgcolor='rgb(0,0,0)', font_size=16)
fig.write_image('results/dynamic-smart.jpg')